# New Digital Design Onboarding Assignment 

The first cell generates two numbers and computes the result. 

The second cell repeats the first cell 128 times and writes the answers to a text file. 



In [5]:

import numpy as np 

'''
Assumptions: 
1. The code generates two 64-bit unsigned integers.
2. Each 64-bit integer is formed by combining strictly two 32-bit integers.
3. Overflow is not handled
'''

mem_value1 = np.random.randint(0, 2**64, size=1, dtype=np.uint64) 
print("64-bit binary number: " + format(mem_value1.item(), '064b'))

# Extract lower 32 bits
lower = mem_value1.item() & 0xFFFFFFFF
print("First 32-bit number: " + bin(lower))

# Extract upper 32 bits
upper = (mem_value1.item() >> 32) & 0xFFFFFFFF
print("First 32-bit number: " + bin(upper))

result1 = lower + upper

mem_value2 = np.random.randint(0, 2**64, size=1, dtype=np.uint64) 
print("64-bit binary number: " + format(mem_value2.item(), '064b'))

# Extract lower 32 bits
lower = mem_value2.item() & 0xFFFFFFFF

# Extract upper 32 bits
upper = (mem_value2.item() >> 32) & 0xFFFFFFFF

result2 = lower + upper

result = result1 + (result2 << 32)

result_binary = format(result, '064b')
print("Result of addition in binary: " + result_binary)

64-bit binary number: 1000110001110010000011111100100000101001001110011100110001110100
First 32-bit number: 0b101001001110011100110001110100
First 32-bit number: 0b10001100011100100000111111001000
64-bit binary number: 0000000100011110001111111011001101110101110011000100111001001100
Result of addition in binary: 0111011011101010100011011111111110110101101010111101110000111100


In [6]:
import numpy as np



with open("output_128_results.txt", "w") as f:
    for i in range(128):
        # Generate first 64-bit integer
        value1 = np.random.randint(0, 2**32, size=1, dtype=np.uint64)
        value1 = value1 << 32
        value1 = value1 | np.random.randint(0, 2**32, size=1, dtype=np.uint64)  
        combined_binary1 = format(value1.item(), '064b')

        # Generate second 64-bit integer
        value2 = np.random.randint(0, 2**32, size=1, dtype=np.uint64)
        value2 = value2 << 32
        value2 = value2 | np.random.randint(0, 2**32, size=1, dtype=np.uint64)  
        combined_binary2 = format(value2.item(), '064b')

        # Add the two 64-bit values
        result = value1 + value2
        result_binary = format(result.item(), '064b')

        # Write the results to the file with an iteration header
        #f.write(f"{i+1}: " + combined_binary1 + " " + combined_binary2 + " " + result_binary + "\n")
        f.write(combined_binary1 + " " + combined_binary2 + " " + result_binary + "\n")


print("Results saved in 'output_128_results.txt'")


Results saved in 'output_128_results.txt'


## Memory sim test script

- To customize, edit the #TODO: below and the instructions.txt for assembly. 
- Since the program only needs to handle addition, instructions.txt does not have a "add" opcode. Instructions are in the format M1 S1 S2, where M1 = S1+S2. 

In [ ]:

import numpy as np

read_start_addr = 0
read_end_addr = 255
write_start_addr = 384
write_end_addr = 511

def gen():
    """Generates a 64-bit unsigned integer."""
    value = np.random.randint(0, 2**64, size=1, dtype=np.uint64) 
    return value

# Generate and write pre-state memory
with open("memory_pre_state_upper.txt", "w") as u, open("memory_pre_state_lower.txt", "w") as l:
    for i in range(256):
        value = gen()
        upper = ((value.item() >> 32) & 0xFFFFFFFF)
        lower = (value.item() & 0xFFFFFFFF)
        u.write(format(upper, '032b') + "\n")
        l.write(format(lower, '032b') + "\n")
    for i in range(256):
        u.write(format(0, '032b') + "\n")
        l.write(format(0, '032b') + "\n")

print("Results saved in 'memory_pre_state_upper.txt' and 'memory_pre_state_lower.txt'")

# Simulate addition
with open("memory_post_state_upper.txt", "w") as u_out, open("memory_post_state_lower.txt", "w") as l_out:
    with open("memory_pre_state_upper.txt", "r") as u_in, open("memory_pre_state_lower.txt", "r") as l_in:
        # Read all pre-state lines
        upper_lines = u_in.readlines()
        lower_lines = l_in.readlines()

        read_addr = read_start_addr
        write_addr = write_start_addr
        for i in range(128):
            # Convert binary strings to integers
            src1_upper = int(upper_lines[read_addr].strip(), 2)
            src1_lower = int(lower_lines[read_addr].strip(), 2)
            
            read_addr += 1

            src2_upper = int(upper_lines[read_addr].strip(), 2)
            src2_lower = int(lower_lines[read_addr].strip(), 2)
            
            read_addr += 1

            src1 = (src1_upper + src1_lower) & 0xFFFFFFFF
            src2 = (src2_upper + src2_lower) & 0xFFFFFFFF

            print("src1: " + bin(src1))
            print("src2: " + bin(src2))
            result = (src2 << 32) | src1
            print("result: " + bin(result))
            res_upper = (result >> 32) & 0xFFFFFFFF
            res_lower = result & 0xFFFFFFFF

            # Save result in memory at the correct index
            upper_lines[write_addr] = format(res_upper, '032b') + "\n"
            lower_lines[write_addr] = format(res_lower, '032b') + "\n"

            write_addr += 1

        # Write post-state memory
        u_out.writelines(upper_lines)
        l_out.writelines(lower_lines)

print("Post-state memory written to 'memory_post_state_upper.txt' and 'memory_post_state_lower.txt'")


Results saved in 'memory_pre_state_upper.txt' and 'memory_pre_state_lower.txt'
Post-state memory written to 'memory_post_state_upper.txt' and 'memory_post_state_lower.txt'
